In [1]:
from tkinter import Frame
import pygame
import sys
import random
import time
import os
import math
from pygame.locals import *
pygame.init() 

# SOUND
pygame.mixer.init(44100, -16, 2, 512) 
pygame.mixer.set_num_channels(32)
jump = pygame.mixer.Sound("assets/jump.wav") #load jump sound and assign it to "jump"
jump.set_volume(0.2) #sets the volume to 20%
pygame.mixer.music.load("assets/music.ogg") #load music
explosions = pygame.mixer.Sound("assets/explosion.wav") #load explosion sound and assign it to "explosions"
explosions.set_volume(0.3) #sets the volume to 30%


FramePerSec = pygame.time.Clock() #assign pygame.time.Clock() to FramePersSec

blue = (0,0,255) #Colors
red = (255,0,0) #Colors
green = (0,255,0) #Colors
black = (0,0,0) #Colors
white = (255,255,255) #Colors

sysfont = pygame.font.get_default_font() #assign the default sys font as sysfont
font = pygame.font.SysFont(sysfont,100)  #(font,size)
font2 = pygame.font.SysFont(sysfont, 50) #(font,size)

highscore = [] #empty highscore list
quittext = font2.render("Quit", True,black)
playtext = font2.render("Play again?", True,black)
screen_w = 1920 #assign our game screen width to variable
screen_h = 1080 #assign our game screen height to variable
screen = pygame.display.set_mode((screen_w, screen_h)) #create a screen based on our variables
pygame.display.set_caption("Älgy The Moose") #sets the name of the screen
explosion = pygame.image.load("assets/explosion.png").convert_alpha() #import image and assigns it and changes the pixel format

bg = pygame.image.load("assets/pattern.jpg").convert() #import image and assigns it and changes the pixel format
bg2 = pygame.image.load("assets/pattern2.jpg").convert() #import image and assigns it and changes the pixel format
bg3 = pygame.image.load("assets/pattern3.jpg").convert() #import image and assigns it and changes the pixel format
bg4 = pygame.image.load("assets/pattern4.jpg").convert() #import image and assigns it and changes the pixel format
bg5 = pygame.image.load("assets/pattern5.jpg").convert() #import image and assigns it and changes the pixel format
class Enemy(pygame.sprite.Sprite): #define class Enemy
    def __init__(self): #define the initial function. self = class
        super().__init__()
        self.image = pygame.image.load("assets/Enemy.png").convert_alpha() #import image and assign it to enemy.image
        self.rect = self.image.get_rect() #get the rectangle (hitbox) based on the dimensions of enemy.image
        self.rect.topleft = (2516, 700) #starting position

    def move(self): #define the move function
        self.rect.move_ip(-15,0) #how much enemy moves per update
        if (self.rect.left<-400): #if enemy is offscreen
            self.rect.topleft = (2516,random.randint(300,1020)) #move it back to defined x position and random y position

    def draw(self, surface): #define the draw image
      surface.blit(self.image, self.rect) #draw the image

class Enemy2(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("assets/Enemy.png").convert_alpha()
        self.rect = self.image.get_rect()
        self.rect.bottomleft = (2516, 380)

    def move(self,x):
        self.rect.move_ip(-15,0)
        if (self.rect.left<-400):
            if x <=400:
                self.rect.bottomleft = (2516,180)
            else:
                self.rect.bottomleft = (2516, x - random.randint(750,850)) #Position based on Enemy1. 

    def draw(self, surface):
      surface.blit(self.image, self.rect)

class Coin(pygame.sprite.Sprite): #define the coin class
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("assets/point.png").convert_alpha()
        self.rect = self.image.get_rect()
        self.rect.bottomleft = (1900, random.randint(100,980))

    def move(self,x=1): #Set a random default position other than 3116, otherwise it freaks out
        self.rect.move_ip(-15,0)   
        if (self.rect.left<-400): # If it goes 400 out of screen
            self.rect.top = 0
            self.rect.bottomleft = (2516,random.randint(100,980)) #Resets position
        if x == 3116: # If player touches it, more info on line 339. We need to account for the players placement which is 600 pixels more to the right
            self.rect.bottomleft=(x,random.randint(100,980)) #Resets position, to 3116 on x axis.


    def draw(self, surface):
        surface.blit(self.image, self.rect)

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("assets/Player.png").convert_alpha()
        self.rect =  self.image.get_rect()
        self.rect.inflate_ip(-25,-25) #reduces hitbox
        self.rect.bottomright = (200,(screen_h/2)) #sets start pos

    def update(self):
        pressed_keys = pygame.key.get_pressed() #assign function to variable
        if self.rect.bottom <= 1000: 
            self.rect.move_ip(0,10)

        if self.rect.top >= -20 and self.rect.bottom <= screen_h+2:
            if pressed_keys[K_SPACE]:
                self.rect.move_ip(0, -20)
        elif self.rect.top <-20: #if 
            self.rect.move_ip(0,10)
        elif self.rect.bottom > screen_h+2:
            self.rect.move_ip(0,-10)

    def draw(self, surface):
        surface.blit(self.image, self.rect)

def play(sound):
    pygame.mixer.Sound.play(sound)
    
def gamestart():
    scroll = 0 #scroll pos for scroll start
    iterationsx = 100 #iterations for opacity of bg. 0 = 0%, 100 = 100%
    coinsx = 0 #number of total coins
    coins  = 0 #updates each time it collects a coin, and resets at 5 coins
    flagch = False     #flagcheck so functions only triggers once in loop
    flagch2 = True
    tiles = math.ceil(screen_w  /bg.get_width()) + 1 #gets scale of screen compared to background then rounds up and adds 1. So we know how many tiles we need simoustanely. 
                                                     #For example, if you have 1920 width on screen and have a 480 width on background, we would need 4 (to cover the width) + 1 (to cover transition)
    PC = Player()  #assigns the class to a variable
    EC = Enemy()
    EC2 = Enemy2()
    C1 = Coin()
    points = pygame.sprite.Group() #create group
    enemies = pygame.sprite.Group()

    enemies.add(EC) #add EC to enemies group
    enemies.add(EC2)
    points.add(C1)

    pygame.mixer.music.play() #plays previously defined music

    while True:
        i = 0 #variable to makes sure our loop length is = titles
        for event in pygame.event.get(): #allows the game to quit, needed to run
            if event.type == pygame.QUIT:
                quit()

        scroll -= 1 #Scrollspeed for background (minus 1 for each iteration)
        #CHANGING AND SCROLLING BACKGROUNDS
        transpeed = 5 # speed of opacity change
        #---------------------------------------------------------------------------
        
        if iterationsx >= 100 and coins==0 and flagch is True:
            iterationsx = 0
            flagch = False
            flagch2 = True

        if iterationsx >= 100 and coins==1 and flagch2 is True: #the flagch makes sure this function is only ran on the first iteration of the loop
            iterationsx = 0 # resets iteration
            flagch = False 
            flagch2 = False

        if coins==1:
            iterationsx +=transpeed
            bg2.set_alpha(iterationsx)

        if iterationsx >= 100 and coins==2 and flagch is False:
            iterationsx = 0
            flagch = True 

        if coins ==2:
            iterationsx +=transpeed
            bg3.set_alpha(iterationsx)

        if iterationsx >= 100 and coins==3 and flagch is True:
            iterationsx = 0
            flagch = False 
        if coins ==3:
            iterationsx +=transpeed
            bg4.set_alpha(iterationsx)

        if iterationsx >= 100 and coins==4 and flagch is False:
            iterationsx = 0
            flagch = True 

        if coins ==4:
            iterationsx +=transpeed
            bg5.set_alpha(iterationsx)

        

        if coins==0:
            iterationsx +=transpeed
            bg.set_alpha(iterationsx)
        # ---------------------------------------------------------------------------
        
        
        # SECONDARY
        if iterationsx < 100 and coins ==1:
            while (i<tiles):
                screen.blit(bg, (bg.get_width()*i + scroll, 0)) #if alpha is under 100%, keep showing the previous background.
                #Also where it should blit the backgrounds. So in this case, it will blit them at x axis -1, 1999. Then next loop it will be -2. 1998, etc
                i+=1

        if iterationsx < 100 and coins ==2 and flagch:
            while (i<tiles):
                screen.blit(bg2, (bg2.get_width()*i + scroll, 0))
                i+=1
        if iterationsx < 100 and coins ==3:
            while (i<tiles):
                screen.blit(bg3, (bg3.get_width()*i + scroll, 0))
                i+=1
       
        if iterationsx < 100 and coins ==4:
            while (i<tiles):
                screen.blit(bg4, (bg4.get_width()*i + scroll, 0))
                i+=1
        if iterationsx < 100 and coins ==0:
            while(i<tiles):
                screen.blit(bg5, (bg5.get_width()*i + scroll, 0)) #where it should blit the backgrounds. -1, 1999. Then next loop it will be -2. 1998, etc Until -2000 then it resets
                i+=1
        
        i = 0 #This makes it lag a little lol
        # ---------------------------------------------------------------------------
        #PRIMARY
        if coins ==0: #if 0 coins, use bg as background
             while(i<tiles):
                screen.blit(bg, (bg.get_width()*i + scroll, 0)) #where it should blit the backgrounds. -1, 1999. Then next loop it will be -2. 1998, etc Until -2000 then it resets
                i+=1

        if coins ==1: #if 1 coins, use bg2 as background
            while(i<tiles):
                screen.blit(bg2, (bg2.get_width()*i + scroll, 0))
                i+=1
           
        if coins ==2:
            while(i<tiles):
                screen.blit(bg3, (bg3.get_width()*i + scroll, 0))
                i+=1                
        
        if coins ==3:
            while(i<tiles):
                screen.blit(bg4, (bg4.get_width()*i + scroll, 0))
                i+=1     
            
        if coins ==4:
            while(i<tiles):
                screen.blit(bg5, (bg5.get_width()*i + scroll, 0))
                i+=1    
           
        if abs(scroll) > bg.get_width(): # if a tile has finished "scrolling". Reset scroll to 0. Abs needed due to number being a negative, because we are scrolling to elft   #???????
            scroll = 0  
        # ---------------------------------------------------------------------------

        FramePerSec.tick(60) #Limits fps to 60 per sec
        EC.move() #Moves the bottom enemy
        EC1ps = EC.rect.center[1] #Gets the Y position of bottom enemy
        EC2.move(EC1ps) #Move the top enemy, but with the Y position being influenced by bottom enemy
        C1.move() #Move coin
        PC.update() #Get the input from player
        PC.draw(screen) #Blit the PC
        EC.draw(screen) # Blit EC
        EC2.draw(screen) #Blit EC
        C1.draw(screen) #Blit C1
        exps = PC.rect.topleft
        textsurface2 = font.render(str(coinsx),True,black)
        if pygame.sprite.spritecollideany(PC,enemies):
            play(explosions)
            screen.blit(explosion, (exps))
            pygame.mixer.music.stop()
            highscore.append(coinsx)
            maxhighscore = max(highscore)
            textsurface5 = font.render(f"HIGH SCORE: {maxhighscore}",True,red)
            textsurface3 = font.render(f"Score {coinsx}",True,red)

            pygame.draw.rect(screen,black,[960-176,538,260,70])

            if maxhighscore > 9: #Makes sure that the high score rectangle is smaller if single digit
                pygame.draw.rect(screen,black,[960-300,216,560,60])
            if maxhighscore < 10: # And bigger if double digit
                pygame.draw.rect(screen,black,[960-300,216,520,60])
            
            #Should probably add for triple digits
            
            if coinsx >9:
                pygame.draw.rect(screen,black,[960-176,538,300,70])
            if coinsx <10:
                pygame.draw.rect(screen,black,[960-176,538,260,70])



            screen.blit(textsurface3,(960-170,540)) #Blits the score
            screen.blit(textsurface5,(960-300,216)) #Blits the highscore
            while True: # Stay in death screen until the user either quits or plays again
                mouse = pygame.mouse.get_pos() # Assigns the mouse position to the variable mouse
                for ev in pygame.event.get(): #Get the events that occurs in the current loop
                    if ev.type == pygame.MOUSEBUTTONDOWN: #If that event is mouse down
                        if 640 <= mouse[0] <= 640+100 and 810 <= mouse[1] <= 810+40: #And is within this
                            pygame.quit()
                        if 1070 <= mouse[0] <= 1070+210 and 810 <= mouse[1] <= 810+40: #And is within this
                            gamestart()  
                if 640 <= mouse[0] <= 640+100 and 810 <= mouse[1] <= 810+40: #QUIT
                    pygame.draw.rect(screen,red,[640,810,100,40]) #Changes the color of the rectangle if mouse is within the coordinates
                else:
                    pygame.draw.rect(screen,green,[640,810,100,40]) #Otherwise stay this color

                if 1070 <= mouse[0] <= 1070+210 and 810 <= mouse[1] <= 810+40: #PLAY AGAIN 
                    pygame.draw.rect(screen,blue,[1070,810,210,40]) #Changes the color of the rectangle if mouse is within the coordinates
                else:
                    pygame.draw.rect(screen,green,[1070,810,210,40]) #Otherwise stay this color                                               
                screen.blit(quittext , (640+9,810+5)) #Blit the "Quit" onto screen
                screen.blit(playtext , (1070+7,810+5)) #Blit the "Play Again" onto screen
                pygame.display.update() #Apply changes applied in this loop
        if pygame.sprite.spritecollideany(PC,points): # If player touches coin
            C1.move(3116) #Reset position of coin (Extra value here to account for the difference between -400 and +200. 2516 + 600
            coinsx+=1 #The actual value for score
            coins+=1 #The value for background change, resets at 5
            if coins == 5:
                coins = 0
            play(jump) #Jump lol
        screen.blit(textsurface2,(100,100)) #Blit the score text
        pygame.display.update() #Show all changes done in this loop to the display
gamestart()


pygame 2.1.2 (SDL 2.0.18, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


: 

: 